In [1]:
# 환경 세팅
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

### Merge를 위한 데이터 불러오기 **(경로에 주의할 것)**

In [2]:
# 각 폴더명 딕셔너리
folder_name = {1:"1.회원정보", 2:"2.신용정보", 3:"3.승인매출정보", 4:"4.청구입금정보", 5:"5.잔액정보", 6:"6.채널정보", 7:"7.마케팅정보", 8:"8.성과정보"}

# 현재 디렉터리
HOME = os.getcwd()

# 폴더 리스트를 생성하고 폴더 경로들을 리스트로 저장하기
folder_list = []
for value in folder_name.values():
    folder_list.append(os.path.join(HOME, "data", "train", value))
print(len(folder_list))

# 폴더 내 파일 리스트 읽어오기
fileNameList = {}
for i in range(len(folder_list)):
    fileNameList[i+1] = os.listdir(folder_list[i])
print(fileNameList)

# 파일 경로 리스트 생성
filePathList = {}
temp = []
for i in range(len(folder_list)):
    for j in range(len(fileNameList[i+1])):
        temp.append(os.path.join(folder_list[i], fileNameList[i+1][j]))
    filePathList[i+1] = temp
    temp = []


print(filePathList)

8
{1: ['201807_train_회원정보.parquet', '201808_train_회원정보.parquet', '201809_train_회원정보.parquet', '201810_train_회원정보.parquet', '201811_train_회원정보.parquet', '201812_train_회원정보.parquet'], 2: ['201807_train_신용정보.parquet', '201808_train_신용정보.parquet', '201809_train_신용정보.parquet', '201810_train_신용정보.parquet', '201811_train_신용정보.parquet', '201812_train_신용정보.parquet'], 3: ['201807_train_승인매출정보.parquet', '201808_train_승인매출정보.parquet', '201809_train_승인매출정보.parquet', '201810_train_승인매출정보.parquet', '201811_train_승인매출정보.parquet', '201812_train_승인매출정보.parquet'], 4: ['201807_train_청구정보.parquet', '201808_train_청구정보.parquet', '201809_train_청구정보.parquet', '201810_train_청구정보.parquet', '201811_train_청구정보.parquet', '201812_train_청구정보.parquet'], 5: ['201807_train_잔액정보.parquet', '201808_train_잔액정보.parquet', '201809_train_잔액정보.parquet', '201810_train_잔액정보.parquet', '201811_train_잔액정보.parquet', '201812_train_잔액정보.parquet'], 6: ['201807_train_채널정보.parquet', '201808_train_채널정보.parquet', '201809_train_채널정보.parquet',

### 메모리 변환 및 병합을 돕는 함수

In [6]:
# 메모리를 32비트로 변환
def optimize_memory(df):
    for col in df.select_dtypes(include=['number']).columns:
        # 정수형은 가능한 가장 작은 타입으로 (int8, int16 등)
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        # 실수형은 float32로
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# 메인으로 파일을 받을 함수
def create_DataFrame(filePathList, month):
    if month >= 7& month <= 12:
        for i in range(8):

            # 단계 1 용량 줄이기 64bit -> 32bit
            df = pd.read_parquet(filePathList[i+1][month-7],engine = "fastparquet")
            df = optimize_memory(df)
            # 단계 2 병합
            if i == 0:
                temp_df = df.copy()
            else:
                temp_df = pd.merge(temp_df, df, on = ["기준년월","ID"], how = "left")


        return temp_df
    else:
        print("this is out of month")

### 함수를 실행시켜 월별로 저장

In [ ]:
df1 = create_DataFrame(filePathList, 7)
df2 = create_DataFrame(filePathList, 8)
df3 = create_DataFrame(filePathList, 9)
df4 = create_DataFrame(filePathList, 10)
df5 = create_DataFrame(filePathList, 11)
df6 = create_DataFrame(filePathList, 12)

### parquet 파일로 데이터 용량 줄여서 저장하기

In [19]:
df1.to_parquet("credit_preprocessing_month_7.parquet")
df2.to_parquet("credit_preprocessing_month_8.parquet")
df3.to_parquet("credit_preprocessing_month_9.parquet")
df4.to_parquet("credit_preprocessing_month_10.parquet")
df5.to_parquet("credit_preprocessing_month_11.parquet")
df6.to_parquet("credit_preprocessing_month_12.parquet")

### 만들어진 파일의 정보 및 메모리 용량 확인

In [21]:
df1.info()

<class 'pandas.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 858 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float32(57), float64(4), int16(152), int32(141), int8(454), object(50)
memory usage: 756.1+ MB


### 병합한 파일 저장한 것을 다시 불러와서 제대로 저장됬는지 확인하기

In [22]:
df_test= pd.read_parquet('credit_preprocessing_month_7.parquet', engine='fastparquet')
df_test.info()

<class 'pandas.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 858 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float32(57), float64(4), int16(152), int32(141), int8(454), object(50)
memory usage: 756.1+ MB
